<a href="https://colab.research.google.com/github/seismosmsr/machine_learning/blob/main/summarize-chatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# for google colab you have to install this 2 library before run the code
!pip install pypdf2
!pip install openai

In [12]:
import os
import PyPDF2
import re
import openai
import pandas as pd

In [13]:
# Download pdf direcly
# !curl -o paper.pdf https://www.researchgate.net/profile/Jerry-Hatfield/publication/224147080_Climate_Impacts_on_Agriculture_Implications_for_Crop_Production/links/5464caa20cf267ed84f25c1a/Climate-Impacts-on-Agriculture-Implications-for-Crop-Production.pdf?origin=publicationDetail&_sg%5B0%5D=e1expVa__Rtw_KD82tLam-hYn_STxP9wHsv5YoKNi_OUJXMiYZnx5VxqZd1AQinKeazNscyKqsL8KCWfktR7Yg.XYHGPH8EnueYT8xATKpuFytnUpYuj68eKGCHzxCUDNA2CfiPongWbP4ITE6U9BjJyiDiENTpj2JAkp32lHuQ8g&_sg%5B1%5D=L8PLvwS9uHvi4iK4V__WRHlubrifW0EQY_PLF-Jg9T7RXVXsObMlV42GiE983vdrQFJWXXxPweE_KbcT5-8ze04xpO24eI0VIgPc56gNmH2E.XYHGPH8EnueYT8xATKpuFytnUpYuj68eKGCHzxCUDNA2CfiPongWbP4ITE6U9BjJyiDiENTpj2JAkp32lHuQ8g&_iepl=&_rtd=eyJjb250ZW50SW50ZW50IjoibWFpbkl0ZW0ifQ%3D%3D&_tp=eyJjb250ZXh0Ijp7ImZpcnN0UGFnZSI6Il9kaXJlY3QiLCJwYWdlIjoiX2RpcmVjdCIsInBvc2l0aW9uIjoicGFnZUhlYWRlciJ9fQ #change the url with pdf which you wanna download

In [14]:
# Set the string that will contain the summary
pdf_summary_text = ""

# Open the PDF file
pdf_file_path = "paper.pdf"

# Read the PDF file using PyPDF2
pdf_file = open(pdf_file_path, 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)

In [21]:
openai.api_key = "na" #change the api key with yours

In [22]:
import pandas as pd
import os

def merge_csvs_from_directory(directory_path, output_filename, failed_filename):
    all_dataframes = []
    failed_files = []
    differing_structure_files = []

    first_file = True
    columns_of_first_file = None

    for filename in os.listdir(directory_path):
        if filename.endswith('.csv'):
            filepath = os.path.join(directory_path, filename)
            try:
                df = pd.read_csv(filepath)

                # If it's the first file, save its columns
                if first_file:
                    columns_of_first_file = df.columns.tolist()
                    first_file = False

                # If columns don't match, skip this file
                if df.columns.tolist() != columns_of_first_file:
                    differing_structure_files.append(filename)
                else:
                    all_dataframes.append(df)
            except Exception as e:
                failed_files.append({'filename': filename, 'error': str(e)})
                print(f"Failed to read {filename} due to error: {e}")

    # Concatenate all successfully read dataframes with the same structure
    if all_dataframes:
        merged_df = pd.concat(all_dataframes, ignore_index=True)
        merged_df.to_csv(output_filename, index=False)

    # Compile failures and differing structures into a single list
    for file in differing_structure_files:
        failed_files.append({'filename': file, 'error': 'Differing structure'})

    # Write failed files to a separate CSV
    if failed_files:
        failed_df = pd.DataFrame(failed_files)
        failed_df.to_csv(failed_filename, index=False)

# Example usage
# merge_csvs_from_directory('/path/to/csv/directory', 'merged_output.csv', 'failed_files.csv')


In [17]:
import zipfile
import os

def zip_specific_csv_files_in_directory(directory_path, output_zipfile, starting_string):
    with zipfile.ZipFile(output_zipfile, 'w') as zipf:
        for filename in os.listdir(directory_path):
            if filename.startswith(starting_string) and filename.endswith('.csv'):
                filepath = os.path.join(directory_path, filename)
                if os.path.isfile(filepath):  # Ensure it's a file and not a directory
                    zipf.write(filepath, os.path.basename(filepath))  # Only write the base filename

    print(f"CSV files in '{directory_path}' starting with '{starting_string}' zipped to '{output_zipfile}'")

# Example usage


In [18]:
import zipfile
import os

def zip_selected_files(directory_path, output_zipfile, file_list):
    with zipfile.ZipFile(output_zipfile, 'w') as zipf:
        for filename in file_list:
            filepath = os.path.join(directory_path, filename)
            if os.path.isfile(filepath):  # Ensure it's a file and not a directory
                zipf.write(filepath, os.path.basename(filepath))

    print(f"Selected files from '{directory_path}' zipped to '{output_zipfile}'")

# Example usage
# file_list = ['file1.txt', 'file2.csv', 'image.jpg']


In [19]:
import os
import pandas as pd

def is_csv_empty(file_path):
    return os.path.getsize(file_path) == 0

def has_uneven_columns(df):
    max_cols = len(df.columns)
    for index, row in df.iterrows():
        if len(row) != max_cols:
            return True
    return False

def fix_uneven_columns(df):
    max_cols = len(df.columns)
    mask = df.apply(len) != max_cols
    rows_with_issues = df[mask].index.tolist()

    # Drop rows with issues
    df = df.drop(rows_with_issues)
    return df


In [24]:
range(len(pdf_reader.pages))[14:]

range(14, 21)

In [25]:
pdf_summary_text = ""
# Loop through all the pages in the PDF file
for page_num in range(len(pdf_reader.pages))[14:]:
    # Extract the text from the page
    page_text = pdf_reader.pages[page_num].extract_text().lower()
    # print(page_text)
    bad_read = True
    attempts = 0
    while bad_read:
      response = openai.ChatCompletion.create(
                      model="gpt-3.5-turbo",
                      messages=[
                          {"role": "system", "content": "You are a helpful research assistant. Specfically, you are helping research the impact of climate change on global food systems. We are doing a literature review. We want to know what crops and food systems are being impacted by specific climate change hazards. Only respond in the form of comma seperated values (csv). You always return a correctly formatted csv."},
                          {"role": "user", "content": f"Summarize the following text as a csv document. Do not reply with anything except a markdown document. Please identify any specific climate change hazards, what cropping or food systems they effect, where globally the impact will be experienced, the specific quote from the paper (no more than 100 characters), and approximately what magnitude (e.g. high medium low). I need a table of these results. In the quote, please replace any commas with hyphens. Commas will break the csv formatting. Do not do anything that could possibly break csv formatting. Please make sure that text entries do not use commas internal to any text entries in the table. Please only return a csv table of the hazard. The columns should only be: region, cropping_system, impact, magnitude, quote, page_number (you are working on page {page_num}), paragraph_number. Here is the text:{page_text}. Do not return anything but the properly formatted csv. If any row is missing data fill it with text that says 'No Data'. It is of the utmost importance that the response you give is a properly formatted csv."},
                              ],
                                  )
      page_summary = response["choices"][0]["message"]["content"]

      pdf_summary_text+=page_summary + "\n"

      pdf_summary_file = pdf_file_path.replace(os.path.splitext(pdf_file_path)[1], f"_summary.txt")

      with open(pdf_summary_file, "w+") as file:
          file.write(pdf_summary_text)

      pdf_summary_file = pdf_file_path.replace(os.path.splitext(pdf_file_path)[1], f"_{page_num}_summary.csv")

      with open(pdf_summary_file, "w+") as file:
          file.write(page_summary)

      # file_path = 'path_to_your_csv_file.csv'
      attempts = attempts + 1
      # Check if the CSV is empty
      if is_csv_empty(pdf_summary_file):
          print(f"{pdf_summary_file} is empty.")
      else:
          try:
              df = pd.read_csv(pdf_summary_file)

              # If the CSV has uneven columns, fix it
              if has_uneven_columns(df):
                  print("Fixing uneven columns...")
                  df = fix_uneven_columns(df)
                  df.to_csv(pdf_summary_file, index=False)  # overwrite the original file with the fixed data
                  bad_read = False

              else:
                  print("No issues detected. File remains unchanged.")
                  bad_read = False


          except Exception as e:
              print(f"Error reading file {pdf_summary_file}: {e}")
              print(f"Attempt {attempts} failed.")
              bad_read = True





# Close the PDF file
pdf_file.close()

with open(pdf_summary_file, "r") as file:
    print(file.read())

No issues detected. File remains unchanged.
No issues detected. File remains unchanged.
No issues detected. File remains unchanged.
Error reading file paper_17_summary.csv: Error tokenizing data. C error: Expected 7 fields in line 7, saw 8

Attempt 1 failed.
Error reading file paper_17_summary.csv: Error tokenizing data. C error: Expected 7 fields in line 27, saw 9

Attempt 2 failed.
Error reading file paper_17_summary.csv: Error tokenizing data. C error: Expected 7 fields in line 30, saw 8

Attempt 3 failed.
Error reading file paper_17_summary.csv: Error tokenizing data. C error: Expected 7 fields in line 25, saw 8

Attempt 4 failed.
No issues detected. File remains unchanged.
No issues detected. File remains unchanged.
No issues detected. File remains unchanged.
No issues detected. File remains unchanged.
region,cropping_system,impact,magnitude,quote,page_number,paragraph_number
No Data,No Data,No Data,No Data,No Data,No Data,No Data


In [26]:
merge_csvs_from_directory('/content/', 'merged_output.csv',failed_filename='failed_files.csv')

In [27]:
zip_selected_files('/content/', 'results_3.zip', ['paper.pdf','merged_output.csv','failed_files.csv','csv_files.zip'])


Selected files from '/content/' zipped to 'results_3.zip'
